## Install Open Refine Client API and put it in the active directory

In [1]:
!git clone https://github.com/PaulMakepeace/refine-client-py

fatal: destination path 'refine-client-py' already exists and is not an empty directory.


In [2]:
!cp -R refine-client-py/google ./

In [3]:
!pip install urllib2_file --target ./

Target directory /Volumes/HD-500GB/Users/nikolausn/Documents/msimsvn/msimrepo/Spring2019/PythonOpenRefine/urllib2_file-0.2.1.dist-info already exists. Specify --upgrade to force replacement.
Target directory /Volumes/HD-500GB/Users/nikolausn/Documents/msimsvn/msimrepo/Spring2019/PythonOpenRefine/urllib2_file.py already exists. Specify --upgrade to force replacement.
You are using pip version 9.0.3, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [37]:
!pip install pandas==0.20.3 --target ./

    100% |████████████████████████████████| 14.7MB 41kB/s  eta 0:00:01    91% |█████████████████████████████▍  | 13.5MB 23.7MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/61/28/1d3920e4d1d50b19bc5d24398a7cd85cc7b9a75a490570d5a30c57622d34/pytz-2018.9-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/0e/65/a27186c1692901f7b451572857f6d8d0031b6928500fa479c30a489afeed/numpy-1.16.2-cp35-cp35m-macosx_10_6_intel.macosx_10_9_intel.macosx_10_9_x86_64.macosx_10_10_intel.macosx_10_10_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/41/17/c62faccbfbd163c7f57f3844689e3a78bae1f403648a6afb1d0866d87fbb/python_dateutil-2.8.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl
You are using pip version 9.0.3, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' comma

In [4]:
from google.refine import refine
import pandas as pd

## Check OpenRefine Server connection

In [6]:
refine.RefineServer().get_version()

URLError: <urlopen error [Errno 61] Connection refused for http://127.0.0.1:3333. No Refine server reachable/running; ENV set?>

In [19]:
t = refine.Refine(refine.RefineServer())

## List OpenRefine Projects using list_projects() method

In [22]:
pd.DataFrame(t.list_projects()).transpose()

,contributors,created,creator,customMetadata,description,homepage,image,importOptionMetadata,license,modified,name,rowCount,subject,tags,title,version
1503997082560,,2019-03-19T22:54:43Z,,{},,,,NaN,,2016-12-27T15:15:02Z,IP_publications_2015Aug28 xls,0,,[],,
1611470445474,,2019-03-19T22:54:43Z,,{},,,,NaN,,2017-06-03T02:29:39Z,menu_question csv,0,,[],,
1627151086685,,2019-03-19T22:54:42Z,,{},,,,NaN,,2016-12-27T18:24:20Z,HBV_Genbank_metadata xls,0,,[],,
1640253865572,,2019-03-19T22:54:42Z,,{},,,,"[{u'processQuotes': True, u'fileSource': u'Air...",,2019-02-21T01:52:19Z,Airbnblistings_dirty csv,7594,,[],,
1684046182555,,2019-03-19T22:54:43Z,,{},,,,"[{u'processQuotes': True, u'fileSource': u'Air...",,2019-03-07T01:42:36Z,Airbnblistings_dirty csv,7594,,[],,
1724853533027,,2019-03-19T22:54:43Z,,{},,,,NaN,,2016-09-13T16:49:19Z,IP_publications_2015Aug28 xls,0,,[],,
1734568039091,,2019-03-19T22:54:43Z,,{},,,,NaN,,2016-11-22T01:58:50Z,testTraffic2,0,,[],,
1751164018924,,2019-03-19T22:54:43Z,,{},,,,NaN,,2017-01-11T15:12:51Z,menu_question csv,0,,[],,
1798295202296,,2019-03-19T22:55:43Z,,{},,,,"[{u'fileSource': u'drugsComTest_raw.tsv', u'en...",,2019-03-19T22:58:37Z,drugs com test,0,,[],,
1892275851398,,2019-03-19T22:54:43Z,,{},,,,"[{u'processQuotes': True, u'fileSource': u'Air...",,2019-02-21T23:36:39Z,Airbnblistings_dirty csv csv test 2,7594,,[],,


## Create a New Project

In [13]:
# Download file
!wget https://github.com/nikolausn/Data_Cleaning_Python_Exercise/blob/master/drugsComTest_raw.tsv?raw=true -O drugsComTest_raw.tsv

--2019-03-19 17:47:11--  https://github.com/nikolausn/Data_Cleaning_Python_Exercise/blob/master/drugsComTest_raw.tsv?raw=true
Resolving github.com (github.com)... 192.30.253.113, 192.30.253.112
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/nikolausn/Data_Cleaning_Python_Exercise/raw/master/drugsComTest_raw.tsv [following]
--2019-03-19 17:47:12--  https://github.com/nikolausn/Data_Cleaning_Python_Exercise/raw/master/drugsComTest_raw.tsv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/nikolausn/Data_Cleaning_Python_Exercise/master/drugsComTest_raw.tsv [following]
--2019-03-19 17:47:12--  https://raw.githubusercontent.com/nikolausn/Data_Cleaning_Python_Exercise/master/drugsComTest_raw.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.184.133
Connecting to raw.github

In [25]:
drug_project = t.new_project(project_file="drugsComTest_raw.tsv",project_name="drugs com test",separator="\t")

In [33]:
# number of columns
len(drug_project.columns)

7

In [26]:
drug_project.columns

[u'Column',
 u'drugName',
 u'condition',
 u'review',
 u'rating',
 u'date',
 u'usefulCount']

In [31]:
rows = drug_project.get_rows()

In [32]:
# total rows
rows.total

53766

In [43]:
pd.DataFrame(rows.rows.rows_response)

,cells,flagged,i,starred
0,"[{u'v': 163740}, {u'v': u'Mirtazapine'}, {u'v'...",False,0,False
1,"[{u'v': 206473}, {u'v': u'Mesalamine'}, {u'v':...",False,1,False
2,"[{u'v': 159672}, {u'v': u'Bactrim'}, {u'v': u'...",False,2,False
3,"[{u'v': 39293}, {u'v': u'Contrave'}, {u'v': u'...",False,3,False
4,"[{u'v': 97768}, {u'v': u'Cyclafem 1 / 35'}, {u...",False,4,False
5,"[{u'v': 208087}, {u'v': u'Zyclara'}, {u'v': u'...",False,5,False
6,"[{u'v': 215892}, {u'v': u'Copper'}, {u'v': u'B...",False,6,False
7,"[{u'v': 169852}, {u'v': u'Amitriptyline'}, {u'...",False,7,False
8,"[{u'v': 23295}, {u'v': u'Methadone'}, {u'v': u...",False,8,False
9,"[{u'v': 71428}, {u'v': u'Levora'}, {u'v': u'Bi...",False,9,False


In [58]:
pd.DataFrame([x["cells"] for x in rows.rows.rows_response],columns=drug_project.columns)

,Column,drugName,condition,review,rating,date,usefulCount
0,{u'v': 163740},{u'v': u'Mirtazapine'},{u'v': u'Depression'},"{u'v': u'""I&#039;ve tried a few antidepressant...",{u'v': 10},"{u'v': u'February 28, 2012'}",{u'v': 22}
1,{u'v': 206473},{u'v': u'Mesalamine'},"{u'v': u'Crohn's Disease, Maintenance'}","{u'v': u'""My son has Crohn&#039;s disease and ...",{u'v': 8},"{u'v': u'May 17, 2009'}",{u'v': 17}
2,{u'v': 159672},{u'v': u'Bactrim'},{u'v': u'Urinary Tract Infection'},"{u'v': u'""Quick reduction of symptoms""'}",{u'v': 9},"{u'v': u'September 29, 2017'}",{u'v': 3}
3,{u'v': 39293},{u'v': u'Contrave'},{u'v': u'Weight Loss'},"{u'v': u'""Contrave combines drugs that were us...",{u'v': 9},"{u'v': u'March 5, 2017'}",{u'v': 35}
4,{u'v': 97768},{u'v': u'Cyclafem 1 / 35'},{u'v': u'Birth Control'},"{u'v': u'""I have been on this birth control fo...",{u'v': 9},"{u'v': u'October 22, 2015'}",{u'v': 4}
5,{u'v': 208087},{u'v': u'Zyclara'},{u'v': u'Keratosis'},"{u'v': u'""4 days in on first 2 weeks. Using o...",{u'v': 4},"{u'v': u'July 3, 2014'}",{u'v': 13}
6,{u'v': 215892},{u'v': u'Copper'},{u'v': u'Birth Control'},"{u'v': u'""I&#039;ve had the copper coil for ab...",{u'v': 6},"{u'v': u'June 6, 2016'}",{u'v': 1}
7,{u'v': 169852},{u'v': u'Amitriptyline'},{u'v': u'Migraine Prevention'},"{u'v': u'""This has been great for me. I&#039;v...",{u'v': 9},"{u'v': u'April 21, 2009'}",{u'v': 32}
8,{u'v': 23295},{u'v': u'Methadone'},{u'v': u'Opiate Withdrawal'},"{u'v': u'""Ive been on Methadone for over ten y...",{u'v': 7},"{u'v': u'October 18, 2016'}",{u'v': 21}
9,{u'v': 71428},{u'v': u'Levora'},{u'v': u'Birth Control'},"{u'v': u'""I was on this pill for almost two ye...",{u'v': 2},"{u'v': u'April 16, 2011'}",{u'v': 3}


In [53]:
 [x["cells"] for x in rows.rows.rows_response]

[[{u'v': 163740},
  {u'v': u'Mirtazapine'},
  {u'v': u'Depression'},
  {u'v': u'"I&#039;ve tried a few antidepressants over the years (citalopram, fluoxetine, amitriptyline), but none of those helped with my depression, insomnia &amp; anxiety. My doctor suggested and changed me onto 45mg mirtazapine and this medicine has saved my life. Thankfully I have had no side effects especially the most common - weight gain, I&#039;ve actually lost alot of weight. I still have suicidal thoughts but mirtazapine has saved me."'},
  {u'v': 10},
  {u'v': u'February 28, 2012'},
  {u'v': 22}],
 [{u'v': 206473},
  {u'v': u'Mesalamine'},
  {u'v': u"Crohn's Disease, Maintenance"},
  {u'v': u'"My son has Crohn&#039;s disease and has done very well on the Asacol.  He has no complaints and shows no side effects.  He has taken as many as nine tablets per day at one time.  I&#039;ve been very happy with the results, reducing his bouts of diarrhea drastically."'},
  {u'v': 8},
  {u'v': u'May 17, 2009'},
  {u'v'

In [55]:
drug_project.compute_facets

In [69]:
review_cluster = drug_project.compute_clusters("review")

In [72]:
drug_project.compute_clusters("review")

[[{'count': 2, 'value': u'"Love it."'},
  {'count': 1, 'value': u'" love it"'},
  {'count': 1, 'value': u'"Love It Love It!!"'},
  {'count': 1, 'value': u'"Love it"'},
  {'count': 1, 'value': u'"love it"'}],
 [{'count': 6, 'value': u'"Works great"'},
  {'count': 3, 'value': u'"Works great!"'},
  {'count': 2, 'value': u'"Works great."'},
  {'count': 1, 'value': u'"Works great. "'},
  {'count': 1, 'value': u'"works great"'}],
 [{'count': 8, 'value': u'"Good."'},
  {'count': 6, 'value': u'"Good"'},
  {'count': 2, 'value': u'"good"'},
  {'count': 1, 'value': u'"Good "'}],
 [{'count': 7, 'value': u'"Excellent"'},
  {'count': 2, 'value': u'"Excellent!!"'},
  {'count': 2, 'value': u'"Excellent!"'},
  {'count': 2, 'value': u'"Excellent."'}],
 [{'count': 5, 'value': u'"Very good"'},
  {'count': 2, 'value': u'"Very good."'},
  {'count': 1, 'value': u'"Very good!"'},
  {'count': 1, 'value': u'"very good"'}],
 [{'count': 1, 'value': u'"Works for me!"'},
  {'count': 1, 'value': u'"Works for me"'},


In [61]:
?drug_project.mass_edit

In [71]:
drug_project.mass_edit("review",edits=[{'from':[x["value"] for x in review_cluster[0]],'to':"it works"}])

{u'code': u'ok',
 u'historyEntry': {u'description': u'Mass edit 11 cells in column review',
  u'id': 1553039579970,
  u'time': u'2019-03-19T23:49:53Z'}}

In [75]:
drug_project.add_column("review","review_new")

{u'code': u'ok',
 u'historyEntry': {u'description': u'Create new column review_new based on column review by filling 53766 rows with value',
  u'id': 1553040608337,
  u'time': u'2019-03-19T23:53:43Z'}}

In [77]:
drug_project.text_transform("review","value.toLowercase()")

{u'code': u'ok',
 u'historyEntry': {u'description': u'Text transform on 53533 cells in column review: value.toLowercase()',
  u'id': 1553040189267,
  u'time': u'2019-03-19T23:55:55Z'}}

In [81]:
drug_project.history_entry.description

u'Text transform on 53533 cells in column review: value.toLowercase()'

In [83]:
drug_project.u

u'2019-03-19T23:55:55Z'